In [2]:
import zarr
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [54]:
def build_mission_root(dataset_folder, mission):
    base = Path(dataset_folder) / mission / "data"
    mission_root = {}

    for d in sorted(base.iterdir()):
        if d.is_dir() and (d / ".zgroup").exists():
            try:
                g = zarr.open_group(store=zarr.storage.LocalStore(str(d)), mode="r")
                mission_root[d.name] = g
            except Exception as e:
                print(f"[skip] {d.name}: {e}")

    return mission_root

dataset_folder = Path("~/grand_tour_dataset/missions").expanduser()
mission = "2024-10-01-11-47-44"

mission_root = build_mission_root(dataset_folder, mission)

print("Available groups:", list(mission_root.keys()))
print("Arrays in adis_imu:", list(mission_root["adis_imu"].keys()))


Available groups: ['adis_imu', 'alphasense_front_center', 'alphasense_front_left', 'alphasense_front_right', 'alphasense_imu', 'alphasense_left', 'alphasense_right', 'anymal_command_twist', 'anymal_imu', 'anymal_state_actuator', 'anymal_state_battery', 'anymal_state_odometry', 'anymal_state_state_estimator', 'ap20_imu', 'cpt7_imu', 'depth_camera_front_upper', 'depth_camera_left', 'depth_camera_rear_upper', 'depth_camera_right', 'dlio_hesai_points_undistorted', 'dlio_map_odometry', 'dlio_tf', 'hdr_front', 'hdr_left', 'hdr_right', 'hesai', 'hesai_points', 'hesai_points_undistorted', 'hesai_undistorted', 'livox_imu', 'livox_points', 'livox_points_undistorted', 'prism_position', 'stim320_accelerometer_temperature', 'stim320_gyroscope_temperature', 'stim320_imu', 'tf', 'zed2i_depth_confidence_image', 'zed2i_depth_image', 'zed2i_left_images', 'zed2i_right_images', 'zed2i_vio_map']
Arrays in adis_imu: ['ang_vel', 'lin_acc', 'ang_vel_cov', 'timestamp', 'orien_cov', 'lin_acc_cov', 'orien', 'seq

In [55]:
def print_group(name,group):
    print( "Zarr group: ", name )

    print(f"\n\033[1mData Fields\033[0m")
    for k in group.keys():

        print(f"\n\033[1mAccessing Data for `{k}`:\033[0m")
        v = group[k]
        print("    - shape: ", v.shape)
        print("    - dtype: ", v.dtype)
        print("    - type:  ", type(v))
        print("    - value: ", v[0], " for idx [0]")


    print("\n\033[1mAttributes:\033[0m")   
    for k,v in group.attrs.items():
        print(f"    - {k}:     ", v)



In [56]:
print_group('alphasense_imu',mission_root['alphasense_imu'])

Zarr group:  alphasense_imu

Data Fields

Accessing Data for `ang_vel`:
    - shape:  (86300, 3)
    - dtype:  float64
    - type:   <class 'zarr.core.array.Array'>
    - value:  [-0.00724402 -0.00426119  0.00340895]  for idx [0]

Accessing Data for `ang_vel_cov`:
    - shape:  (86300, 3, 3)
    - dtype:  float64
    - type:   <class 'zarr.core.array.Array'>
    - value:  [[8.16316965e-06 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 8.16316965e-06 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 8.16316965e-06]]  for idx [0]

Accessing Data for `orien`:
    - shape:  (86300, 4)
    - dtype:  float64
    - type:   <class 'zarr.core.array.Array'>
    - value:  [0. 0. 0. 0.]  for idx [0]

Accessing Data for `lin_acc`:
    - shape:  (86300, 3)
    - dtype:  float64
    - type:   <class 'zarr.core.array.Array'>
    - value:  [ 0.2466748  -0.06914551 -9.5819165 ]  for idx [0]

Accessing Data for `sequence_id`:
    - shape:  (86300,)
    - dtype:  uint64
    - type:   <class 'zarr.core.arra

In [57]:
keys =  list( mission_root.keys())
sorted( keys )
bad_keys = []
for k in keys:
    print_group(k,mission_root[k])
    print("--------------------------------------------\n")

Zarr group:  adis_imu

Data Fields

Accessing Data for `ang_vel`:
    - shape:  (84012, 3)
    - dtype:  float64
    - type:   <class 'zarr.core.array.Array'>
    - value:  [-0.00073495 -0.00047392 -0.00102069]  for idx [0]

Accessing Data for `ang_vel_cov`:
    - shape:  (84012, 3, 3)
    - dtype:  float64
    - type:   <class 'zarr.core.array.Array'>
    - value:  [[2.09297367e-07 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 2.09297367e-07 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 2.09297367e-07]]  for idx [0]

Accessing Data for `lin_acc`:
    - shape:  (84012, 3)
    - dtype:  float64
    - type:   <class 'zarr.core.array.Array'>
    - value:  [0.04860438 0.15203792 9.68292087]  for idx [0]

Accessing Data for `lin_acc_cov`:
    - shape:  (84012, 3, 3)
    - dtype:  float64
    - type:   <class 'zarr.core.array.Array'>
    - value:  [[0.66450853 0.         0.        ]
 [0.         0.66450853 0.        ]
 [0.         0.         0.66450853]]  for idx [0]

Accessing Data for 

In [58]:
keys =  list( mission_root.keys())
sorted( keys )
bad_keys = []
for k in keys:
    print(f"Topic/Key: \033[1m{k}\033[0m")
    try:
        print("   Data Fields:     ", sorted(list(mission_root[k].keys())))
        print("   Data Attributes: ", list( mission_root[k].attrs))
        print("   Description:     ", mission_root[k].attrs["description"])
    except Exception as e:
        bad_keys.append(k)
        continue 
    print("")

print(f"Bad Keys: {bad_keys}")

Topic/Key: adis_imu
   Data Fields:      ['ang_vel', 'ang_vel_cov', 'lin_acc', 'lin_acc_cov', 'orien', 'orien_cov', 'sequence_id', 'timestamp']
   Data Attributes:  ['description', 'frame_id', 'topic', 'transform']
   Description:      Analog Devices ADIS16475-2 200Hz

Topic/Key: alphasense_front_center
   Data Fields:      ['sequence_id', 'timestamp']
   Data Attributes:  ['camera_info', 'description', 'frame_id', 'topic', 'transform']
   Description:      10Hz - 1440x1080 - Sevensense Alphasense/CoreResearch front center (Sony IMX-273, Color, 1.6MP)

Topic/Key: alphasense_front_left
   Data Fields:      ['sequence_id', 'timestamp']
   Data Attributes:  ['camera_info', 'description', 'frame_id', 'topic', 'transform']
   Description:      10Hz - 1440x1080 - Sevensense Alphasense/CoreResearch front left (Sony IMX-273, Mono, 1.6MP)

Topic/Key: alphasense_front_right
   Data Fields:      ['sequence_id', 'timestamp']
   Data Attributes:  ['camera_info', 'description', 'frame_id', 'topic', 